Summarizing dialogues

In [1]:
!pip install --upgrade pip
!pip install transformers datasets rouge_score --quiet
!pip install torch --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
# Import necessary libraries
from datasets import load_dataset
from transformers import pipeline
from rouge_score import rouge_scorer

In [3]:
# Load a dialogue summarization model
# Hugging Face's "facebook/bart-large-cnn" pre-trained model as an example.
summarizer = pipeline("summarization", model="google/flan-t5-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


In [4]:
dataset = load_dataset("daily_dialog")
print(dataset['train'][0])  # Inspect a sample

README.md:   0%|          | 0.00/7.27k [00:00<?, ?B/s]

daily_dialog.py:   0%|          | 0.00/4.85k [00:00<?, ?B/s]

The repository for daily_dialog contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/daily_dialog.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/11118 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'dialog': ['Say , Jim , how about going for a few beers after dinner ? ', ' You know that is tempting but is really not good for our fitness . ', ' What do you mean ? It will help us to relax . ', " Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ? ", " I guess you are right.But what shall we do ? I don't feel like sitting at home . ", ' I suggest a walk over to the gym where we can play singsong and meet some of our friends . ', " That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them . ", ' Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too . ', " Good.Let ' s go now . ", ' All right . '], 'act': [3, 4, 2, 2, 2, 3, 4, 1, 3, 4], 'emotion': [0, 0, 0, 0, 0, 0, 4, 4, 4, 4]}


In [27]:
# Prepare the dialogue for summarization (convert list to a single string)
def format_dialogue(dialogue_list):
    return " ".join(dialogue_list)

In [28]:
dialogue1 = format_dialogue(dataset['train'][0]['dialog'])
dialogue2 = format_dialogue(dataset['train'][1]['dialog'])
dialogue3 = format_dialogue(dataset['train'][2]['dialog'])

In [29]:
dialogue1

"Say , Jim , how about going for a few beers after dinner ?   You know that is tempting but is really not good for our fitness .   What do you mean ? It will help us to relax .   Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ?   I guess you are right.But what shall we do ? I don't feel like sitting at home .   I suggest a walk over to the gym where we can play singsong and meet some of our friends .   That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them .   Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too .   Good.Let ' s go now .   All right . "

In [30]:
dialogue2

"Can you do push-ups ?   Of course I can . It's a piece of cake ! Believe it or not , I can do 30 push-ups a minute .   Really ? I think that's impossible !   You mean 30 push-ups ?   Yeah !   It's easy . If you do exercise everyday , you can make it , too . "

In [31]:
dialogue3

"Can you study with the radio on ?   No , I listen to background music .   What is the difference ?   The radio has too many comerials .   That's true , but then you have to buy a record player . "

In [10]:
#Evaluate the model's performance (Zero-Shot Example)
generated_summary = summarizer(dialogue1, max_length=50, min_length=25, do_sample=False)

In [11]:
print("\nModel-Generated Summary (Zero-Shot):")
print(generated_summary[0]['summary_text'])


Model-Generated Summary (Zero-Shot):
Jim and Jim are going for a beer after dinner . They are going to the gym to play ping pong . Jim and Sally are going dancing .


In [12]:
#Evaluate the model's performance on second dialogue (Zero-Shot Example)
generated_summary2 = summarizer(dialogue2, max_length=50, min_length=25, do_sample=False)

In [13]:
print("\nModel-Generated Summary for second dialogue (Zero-Shot):")
print(generated_summary2[0]['summary_text'])


Model-Generated Summary for second dialogue (Zero-Shot):
I think I can do 30 push-ups a minute , but I think it's impossible .


In [14]:
# One-Shot Learning
# Provide one labeled example as context in the prompt
one_shot_dialogue = format_dialogue(dataset['train'][0]['dialog'])

In [15]:
one_shot_prompt = f"""
Example:
Dialogue: {one_shot_dialogue}
Summary: A friendly conversation about daily activities and personal interests.

Now summarize this dialogue:
Dialogue: {dialogue2}
Summary:
"""

In [16]:
one_shot_prompt.split('\n')

['',
 'Example:',
 "Dialogue: Say , Jim , how about going for a few beers after dinner ?   You know that is tempting but is really not good for our fitness .   What do you mean ? It will help us to relax .   Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ?   I guess you are right.But what shall we do ? I don't feel like sitting at home .   I suggest a walk over to the gym where we can play singsong and meet some of our friends .   That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them .   Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too .   Good.Let ' s go now .   All right . ",
 'Summary: A friendly conversation about daily activities and personal interests.',
 '',
 'Now summarize this dialogue:',
 "Dialogue: Can you do push-ups ?   Of course I can . It's a piece of cake ! Believe it or not , I can do 30

In [17]:
one_shot_generated_summary = summarizer(one_shot_prompt, max_length=50, min_length=25, do_sample=False)

In [18]:
print("\nModel-Generated Summary (One-Shot):")
print(one_shot_generated_summary[0]['summary_text'])


Model-Generated Summary (One-Shot):
I can do 30 push-ups a minute . I think that's impossible . If you do exercise everyday , you can make it .


In [19]:
#Few-Shot Learning
few_shot_dialogue_1 = format_dialogue(dataset['train'][0]['dialog'])
few_shot_dialogue_2 = format_dialogue(dataset['train'][1]['dialog'])
few_shot_prompt = f"""
Examples:
Dialogue: {few_shot_dialogue_1}
Summary: A friendly conversation about daily activities and personal interests.

Dialogue: {few_shot_dialogue_2}
Summary: A discussion about fitness and ability to do fast push ups.

Now summarize this dialogue:
Dialogue: {dialogue3}
Summary:
"""

In [20]:
few_shot_generated_summary = summarizer(few_shot_prompt, max_length=50, min_length=5, do_sample=False)
print("\nModel-Generated Summary (Few-Shot):")
print(few_shot_generated_summary[0]['summary_text'])


Model-Generated Summary (Few-Shot):
The radio has too many comerials and you have to buy a record player .


In [21]:
zero_shot_op = summarizer(dialogue3, do_sample=False)

one_shot_prompt = f"""
Example:
Dialogue: {one_shot_dialogue}
Summary: A friendly conversation about daily activities and personal interests.

Now summarize this dialogue:
Dialogue: {dialogue3}
Summary:
"""

one_shot_op = summarizer(one_shot_prompt, do_sample=False)

few_shot_prompt = f"""
Examples:
Dialogue: {few_shot_dialogue_1}
Summary: A friendly conversation about daily activities and personal interests.

Dialogue: {few_shot_dialogue_2}
Summary: A discussion about fitness and ability to do fast push ups.

Now summarize this dialogue:
Dialogue: {dialogue3}
Summary:
"""

few_shot_op = summarizer(few_shot_prompt, do_sample=False)

Your max_length is set to 200, but your input_length is only 57. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=28)


In [22]:
zero_shot_op

[{'summary_text': 'The radio has too many comerials , so you have to buy a record player to listen to the music , and then you can listen to it on the radio .'}]

In [23]:
one_shot_op

[{'summary_text': 'The radio has too many comerials , but you have to buy a record player to listen to them , so you can study with it .'}]

In [24]:
few_shot_op

[{'summary_text': 'The radio has too many comerials , but you have to buy a record player to listen to them , so you can study with it .'}]

In [25]:
# Compare model-generated summaries with hypothetical "ground truth"
hypothetical_summary = "A friendly conversation on studying with background music."

# Initialize the scorer with the desired metrics
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

zero_shot_scores = scorer.score(hypothetical_summary, zero_shot_op[0]['summary_text'])
one_shot_scores = scorer.score(hypothetical_summary, one_shot_op[0]['summary_text'])
few_shot_scores = scorer.score(hypothetical_summary, few_shot_op[0]['summary_text'])

print("\nROUGE Scores (Zero-Shot):", zero_shot_scores)
print("ROUGE Scores (One-Shot):", one_shot_scores)
print("ROUGE Scores (Few-Shot):", few_shot_scores)



ROUGE Scores (Zero-Shot): {'rouge1': Score(precision=0.10344827586206896, recall=0.375, fmeasure=0.16216216216216217), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.06896551724137931, recall=0.25, fmeasure=0.1081081081081081)}
ROUGE Scores (One-Shot): {'rouge1': Score(precision=0.125, recall=0.375, fmeasure=0.1875), 'rouge2': Score(precision=0.043478260869565216, recall=0.14285714285714285, fmeasure=0.06666666666666667), 'rougeL': Score(precision=0.125, recall=0.375, fmeasure=0.1875)}
ROUGE Scores (Few-Shot): {'rouge1': Score(precision=0.125, recall=0.375, fmeasure=0.1875), 'rouge2': Score(precision=0.043478260869565216, recall=0.14285714285714285, fmeasure=0.06666666666666667), 'rougeL': Score(precision=0.125, recall=0.375, fmeasure=0.1875)}


In [26]:

# Example of Prompt Engineering

# Role-based prompt
role_based_prompt = "You are a helpful assistant. Summarize the following dialogue in one sentence:"
dialogue = "Person A: Hey, how are you? Person B: I'm good, just working on a project.Person A: Nice, hope it goes well!"

role_based_summary = summarizer(role_based_prompt + dialogue3, max_length=50, min_length=25, do_sample=False)
print("Role-Based Prompt Summary:")
print(role_based_summary[0]['summary_text'])

# Instructional prompt
instructional_prompt = "Summarize the dialogue below in exactly 10 words:" + dialogue3

instructional_summary = summarizer(instructional_prompt, max_length=50, min_length=10, do_sample=False)
print("Instructional Prompt Summary:")
print(instructional_summary[0]['summary_text'])

# Stylistic prompt
stylistic_prompt = "Summarize the dialogue below as a poem:" + dialogue3

stylistic_summary = summarizer(stylistic_prompt, max_length=50, min_length=10, do_sample=False)
print("Stylistic Prompt Summary:")
print(stylistic_summary[0]['summary_text'])


Role-Based Prompt Summary:
The radio has too many comerials , so you have to buy a record player to listen to them .
Instructional Prompt Summary:
The radio has too many comerials , so you have to buy a record player .
Stylistic Prompt Summary:
The radio has too many comerials , so you have to buy a record player .
